___

# The battle of Neighborhoods in Hyderabad

### Determining where to open a new shopping centre in Hyedrabad, India.
___

## Table of Contents

 <div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#scrapping">Web Scrapping for DataSet</a></li>
        <li><a href="#cord">Getting Geographical Contents</a></li>
        <li><a href="#venue">Obtaining Venue Data Using foursquare API</a></li>
        <li><a href="#cluster">Clustering the Neighborhoods and mapping the clusters</a></li>
        <li><a href="#choose">Choosing the best cluster </a></li>
    </ol>
</div>

**Intaling Required Packages and Dependancies**

In [0]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install beautifulsoup4
!{sys.executable} -m pip install requests

In [0]:
!pip install geocoder

In [0]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


<div id="#scrapping">
    <h2>Web Scrapping of the DataSet</h2>
</div>

- The data is collected from the [wikipedia](https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Hyderabad,_India) page which consists all the neighborhoods of Hyderabad in alphabetical order.

- The data collected is pre processed in BeautifulSoup package of python and then converted to DataFrame


In [0]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Hyderabad,_India").text

In [0]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [0]:
# create a list to store neighborhood data
neighborhoodList = []

In [0]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [0]:
# create a new DataFrame from the list
hyd_df = pd.DataFrame({"Neighborhood": neighborhoodList})

hyd_df.head(10)

,Neighborhood
0,A. S. Rao Nagar
1,A.C. Guards
2,Abhyudaya Nagar
3,Abids
4,Adikmet
5,Afzal Gunj
6,Aghapura
7,"Aliabad, Hyderabad"
8,Alijah Kotla
9,Allwyn Colony


In [0]:
# print the number of rows of the dataframe
hyd_df.shape

(200, 1)

<div id="#cord">
    <h2>Getting Geographical Cordiantes</h2>
</div>

- The geographical coordinates of the neighborhoods are obtained by using the geocoder package

In [0]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Hyderabad, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [0]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in hyd_df["Neighborhood"].tolist() ]

In [0]:
coords

[[17.411200000000065, 78.50824000000006],
 [17.392977027745946, 78.45686724051741],
 [17.337650000000053, 78.56414000000007],
 [17.389800000000037, 78.47658000000007],
 [17.410610000000077, 78.51513000000006],
 [17.37751000000003, 78.48005000000006],
 [17.387364823969637, 78.4669870622138],
 [17.34259000000003, 78.47626000000008],
 [17.36068000000006, 78.47998000000007],
 [17.503370000000075, 78.41602000000006],
 [17.535430000000076, 78.54427000000004],
 [17.385820000000024, 78.51836000000003],
 [17.435350000000028, 78.44861000000003],
 [17.40784000000002, 78.49150000000003],
 [17.385140000000035, 78.44738000000007],
 [17.369170000000054, 78.43683000000004],
 [17.40710000000007, 78.50233000000003],
 [17.372720000000072, 78.49047000000007],
 [17.38897000000003, 78.48681000000005],
 [17.39931000000007, 78.49964000000006],
 [17.339920000000063, 78.54553000000004],
 [17.448510000000056, 78.44924000000003],
 [17.415350000000046, 78.43435000000005],
 [17.38859199570786, 78.47665099785392],
 

In [0]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [0]:
df_coords.head()

,Latitude,Longitude
0,17.411200,78.508240
1,17.392977,78.456867
2,17.337650,78.564140
3,17.389800,78.476580
4,17.410610,78.515130


In [0]:
# merge the coordinates into the original dataframe
hyd_df['Latitude'] = df_coords['Latitude']
hyd_df['Longitude'] = df_coords['Longitude']

In [0]:
# check the neighborhoods and the coordinates
print(hyd_df.shape)
hyd_df.head(10)

(200, 3)


,Neighborhood,Latitude,Longitude
0,A. S. Rao Nagar,17.411200,78.508240
1,A.C. Guards,17.392977,78.456867
2,Abhyudaya Nagar,17.337650,78.564140
3,Abids,17.389800,78.476580
4,Adikmet,17.410610,78.515130
5,Afzal Gunj,17.377510,78.480050
6,Aghapura,17.387365,78.466987
7,"Aliabad, Hyderabad",17.342590,78.476260
8,Alijah Kotla,17.360680,78.479980
9,Allwyn Colony,17.503370,78.416020


The neighborhoods data frame and their respective latitudes and the longitudes data frame are saved to ```hyd_df``` csv file

In [0]:
hyd_df.to_csv("hyd_df.csv", index=False)

**Creating a map of Hyderabad with Neighborhoods superimposed on top.**

In [0]:
# get the coordinates of hyderabad
address = 'Hyderabad, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Hyderabad, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Hyderabad, India 17.38878595, 78.46106473453146.


In [0]:
# create map of Hyderabad using latitude and longitude values
map_hyd = folium.Map(location=[latitude, longitude], zoom_start=11.3)

# add markers to map
for lat, lng, neighborhood in zip(hyd_df['Latitude'], hyd_df['Longitude'], hyd_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_hyd)  
    
map_hyd

In [0]:
# save the map as HTML file
map_hyd.save('map_hyd.html')

<div id="#venue">
    <h2>Obtaining the Venue details using foursqare API</h2>
</div>


In [0]:
# define Foursquare Credentials and Version
CLIENT_ID = 'N1TS3FIUG3ZC3Z4SMBXOADRNCJZU24NW4BTFNXP5RB2HNMOM' # your Foursquare ID
CLIENT_SECRET = '1X3TQHWRWJPVKSWMSW4SNWMOF14LCEUZJZUGZ13DTPNKNHLC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: N1TS3FIUG3ZC3Z4SMBXOADRNCJZU24NW4BTFNXP5RB2HNMOM
CLIENT_SECRET:1X3TQHWRWJPVKSWMSW4SNWMOF14LCEUZJZUGZ13DTPNKNHLC


**Lets get the top 100 venues that are within the radius of 2000 metres**

In [0]:
radius = 2000
LIMIT = 100

venues = []

for lat, longi, neighborhood in zip(hyd_df['Latitude'], hyd_df['Longitude'], hyd_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        longi,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            longi, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [0]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(6891, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,A. S. Rao Nagar,17.4112,78.50824,Bawarchi,17.406369,78.497662,Indian Restaurant
1,A. S. Rao Nagar,17.4112,78.50824,Sudharshan Theatre 35mm,17.406530,78.495150,Movie Theater
2,A. S. Rao Nagar,17.4112,78.50824,Subway,17.404173,78.514950,Sandwich Place
3,A. S. Rao Nagar,17.4112,78.50824,Devi 70 MM,17.406329,78.495409,Movie Theater
4,A. S. Rao Nagar,17.4112,78.50824,Spencer's,17.412592,78.498400,Convenience Store


**Lets Check how many venues are returned for each neighborhood**

In [0]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
A. S. Rao Nagar,24,24,24,24,24,24
A.C. Guards,54,54,54,54,54,54
Abhyudaya Nagar,11,11,11,11,11,11
Abids,79,79,79,79,79,79
Adikmet,22,22,22,22,22,22
Afzal Gunj,46,46,46,46,46,46
Aghapura,56,56,56,56,56,56
"Aliabad, Hyderabad",10,10,10,10,10,10
Alijah Kotla,17,17,17,17,17,17


**Let's find out how many unique categories can be curated from all the returned venues.**

In [0]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 171 uniques categories.


In [0]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Indian Restaurant', 'Movie Theater', 'Sandwich Place',
       'Convenience Store', 'Ice Cream Shop', 'Coffee Shop', 'Café',
       'Asian Restaurant', 'Clothing Store', 'Platform', 'Flea Market',
       'Grocery Store', 'Bakery', 'Hyderabadi Restaurant', 'Juice Bar',
       'Lounge', 'South Indian Restaurant', 'Park', 'Bistro',
       'Science Museum', 'Snack Place', 'Middle Eastern Restaurant',
       'Vegetarian / Vegan Restaurant', 'Stadium',
       'Performing Arts Venue', 'Hotel', 'Hotel Bar',
       'Fast Food Restaurant', 'Pizza Place', 'Mobile Phone Shop',
       'Fried Chicken Joint', 'Soccer Field', 'Department Store',
       'Hookah Bar', 'Electronics Store', 'Fruit & Vegetable Store',
       'Restaurant', 'Shoe Store', 'Food Truck', 'Neighborhood',
       'Chaat Place', 'Diner', 'Burger Joint', 'Dessert Shop',
       'Chinese Restaurant', 'Smoke Shop', 'Breakfast Spot', 'Bar',
       'Shopping Mall', 'Food'], dtype=object)

In [0]:
# check if the results contain "Shopping Mall"
"Shopping Mall" in venues_df['VenueCategory'].unique()

True

**Analyzing Each Neighborhood**

In [0]:
# one hot encoding
hyd_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
hyd_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [hyd_onehot.columns[-1]] + list(hyd_onehot.columns[:-1])
hyd_onehot = hyd_onehot[fixed_columns]

print(hyd_onehot.shape)
hyd_onehot.head()

(6891, 172)


,Neighborhoods,ATM,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Buffet,Burger Joint,Bus Station,Business Service,Butcher,Cafeteria,Café,Campground,Chaat Place,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Hunan Restaurant,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irani Cafe,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Lake,Laser Tag,Light Rail Station,Liquor Store,Lounge,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Office,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Plaza,Pool,Pub,Rajasthani Restaurant,Residential Building (Apartment / Condo),Resort,Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Social Club,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Tea Room,Tech Startup,Temple,Tex-Mex Restaurant,Thai Restaurant,Train Station,Vegetarian / Vegan Restaurant,Wings Joint,Women's Store,Zoo
0,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,A. S. Rao Nagar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.**

In [0]:
hyd_grouped = hyd_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(hyd_grouped.shape)
hyd_grouped

(200, 172)


,Neighborhoods,ATM,Afghan Restaurant,American Restaurant,Andhra Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Buffet,Burger Joint,Bus Station,Business Service,Butcher,Cafeteria,Café,Campground,Chaat Place,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Hunan Restaurant,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irani Cafe,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Lake,Laser Tag,Light Rail Station,Liquor Store,Lounge,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Office,Outdoors & Recreation,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Plaza,Pool,Pub,Rajasthani Restaurant,Residential Building (Apartment / Condo),Resort,Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Social Club,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Tea Room,Tech Startup,Temple,Tex-Mex Restaurant,Thai Restaurant,Train Station,Vegetarian / Vegan Restaurant,Wings Joint,Women's Store,Zoo
0,A. S. Rao Nagar,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.083333,0.000000,0.00,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.291667,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.166667,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.0000,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
1,A.C. Guards,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00

In [0]:
len(hyd_grouped[hyd_grouped["Shopping Mall"] > 0])

78

**Create a new DataFrae for shopping Mall data only.**

In [0]:
hyd_mall = hyd_grouped[["Neighborhoods","Shopping Mall"]]
hyd_mall.head()

,Neighborhoods,Shopping Mall
0,A. S. Rao Nagar,0.000000
1,A.C. Guards,0.000000
2,Abhyudaya Nagar,0.000000
3,Abids,0.012658
4,Adikmet,0.000000


<div id="#cluster">
    <h2>Clustering the Neighborhood and mapping the clusters.</h2>
</div>

Run k-means to clusters the neighborhoods in Hyderabad into 3 clusters.

In [0]:
# set number of clusters
hyd_clusters = 3

hyd_clustering = hyd_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=hyd_clusters, random_state=0).fit(hyd_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 2, 0, 0, 2, 0], dtype=int32)

In [0]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
hyd_merged = hyd_mall.copy()

# add clustering labels
hyd_merged["Cluster Labels"] = kmeans.labels_

In [0]:
hyd_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
hyd_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,A. S. Rao Nagar,0.000000,0
1,A.C. Guards,0.000000,0
2,Abhyudaya Nagar,0.000000,0
3,Abids,0.012658,0
4,Adikmet,0.000000,0


In [0]:

# merge hyd_merged with hyd_df to add latitude/longitude for each neighborhood
hyd_merged = hyd_merged.join(hyd_df.set_index("Neighborhood"), on="Neighborhood")

print(hyd_merged.shape)
hyd_merged.head() # check the last columns!

(200, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,A. S. Rao Nagar,0.000000,0,17.411200,78.508240
1,A.C. Guards,0.000000,0,17.392977,78.456867
2,Abhyudaya Nagar,0.000000,0,17.337650,78.564140
3,Abids,0.012658,0,17.389800,78.476580
4,Adikmet,0.000000,0,17.410610,78.515130


In [0]:
# sort the results by Cluster Labels
print(hyd_merged.shape)
hyd_merged.sort_values(["Cluster Labels"], inplace=True)
hyd_merged

(200, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,A. S. Rao Nagar,0.000000,0,17.411200,78.508240
123,Masab Tank,0.010000,0,17.400930,78.453620
125,Mehdipatnam,0.000000,0,17.392630,78.442190
126,Mettuguda,0.000000,0,17.427740,78.528920
127,"Minister Road, Hyderabad",0.000000,0,17.432743,78.484542
128,Mir Alam Tank,0.000000,0,17.355109,78.454123
129,Miyapur,0.000000,0,17.421020,78.582440
133,Moosarambagh,0.000000,0,17.370770,78.518180
134,Moti Nagar (Hyderabad),0.000000,0,17.453976,78.418140
122,Marredpally,0.000000,0,17.447770,78.508730


**Finally, let's visualize the resulting clusters.**

In [215]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(hyd_clusters)
ys = [i+x+(i*x)**2 for i in range(hyd_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(hyd_merged['Latitude'], hyd_merged['Longitude'], hyd_merged['Neighborhood'], hyd_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [0]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

<div id="#choose">
    <h2>Choosing the best Cluster</h2>
</div>

**Exploratory Data analysis and examination of the clusters**

____
**cluster 0**

- The cluster 0 is the biggest cluster among the given clusters.
- The cluster is dense in the middle i.e it has most shopping centers and markets in the heart of the city.
- Few of the shopping centers in this clusters date back to the Nizams period (1800's - 1940's) which are known for their diversity of the products found over there. 
- Thogh the cluster 0 is dense in the middle it becomes sparse as the cluster grows i.e in the outskirts of the city indicating for a better potential of establishing a shopping center.

In [0]:
hyd_merged.loc[hyd_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,A. S. Rao Nagar,0.000000,0,17.411200,78.508240
123,Masab Tank,0.010000,0,17.400930,78.453620
125,Mehdipatnam,0.000000,0,17.392630,78.442190
126,Mettuguda,0.000000,0,17.427740,78.528920
127,"Minister Road, Hyderabad",0.000000,0,17.432743,78.484542
128,Mir Alam Tank,0.000000,0,17.355109,78.454123
129,Miyapur,0.000000,0,17.421020,78.582440
133,Moosarambagh,0.000000,0,17.370770,78.518180
134,Moti Nagar (Hyderabad),0.000000,0,17.453976,78.418140
122,Marredpally,0.000000,0,17.447770,78.508730


___
**Cluster 1**

- This cluster is the smallest cluster among the cluster and is mosstly concentrated on the outward part of the city

- The spopping centres here are mostly concentrated on the Industrial Market

In [0]:
hyd_merged.loc[hyd_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
75,Jalal Baba Nagar,0.142857,1,17.35442,78.432550
15,Attapur,0.200000,1,17.36917,78.436830
22,Bairamalguda,0.142857,1,17.33992,78.545530
162,Prakash Nagar,0.166667,1,17.36069,78.416340
106,Lingojiguda,0.176471,1,17.35067,78.534040
56,Ferozguda,0.125000,1,17.47412,78.426387
46,Chintalakunta,0.142857,1,17.33862,78.557410
187,Saroornagar,0.111111,1,17.35442,78.539210
83,Karmanghat,0.250000,1,17.34061,78.532580


___
**Cluster 2**

- This cluster is the medium sized cluster and probably the cluster with most modern shopping centers and fancy shopping malls. 

In [0]:
hyd_merged.loc[hyd_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
92,"Kothapet, Hyderabad",0.066667,2,17.368830,78.542290
84,Karwan,0.066667,2,17.379070,78.436680
5,Afzal Gunj,0.043478,2,17.377510,78.480050
95,Kukatpally,0.090909,2,17.487350,78.420870
191,Shahran Market,0.055556,2,17.364490,78.475290
190,Shah-Ali-Banda,0.062500,2,17.357390,78.473200
11,Amberpet,0.052632,2,17.385820,78.518360
80,"Kamala Nagar, Hyderabad",0.045455,2,17.365610,78.533050
8,Alijah Kotla,0.058824,2,17.360680,78.479980
81,Kapra,0.062500,2,17.478378,78.563662


## FInal Observations:
 
 - Most of the shopping malls and the shopping centers are concentrated in the center of the city.
 - The highest number of shopping centres are concentrated in the cluster 0 moderate in cluster 2, and lowest in cluster 1.
 - cluster 0 is the densest among the clusters and the cluster becomes sparse as we move outward towards the city.
 - Cluster 0 closer to the financial district has a very high chance of success rate and good potential to business expansion and guarantees a lot of investments
 - cluster 1 te sallest cluster also has a good potential but not as good as cluster 0.
 - cluster 2 is the most competetive cluster among the clusters and has very low success rate.
 - cluster 2 is also a dense cluster and unlike cluster 0 it is concentrated in the heart of the city having highly competetive markets and offers almost highly impossible growth rate for beginners.

### Thank you Reading this Notebook

This notebook was created by [Sathwik Vadlamani](https://www.linkedin.com/in/sathwik-vadlamani-a14a84169/). I hope you found this notebook interesting and educational. Feel free to contact if you have any questions!

This notebook is part of a course on **Coursera** called *Applied Data Science Capstone*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://www.coursera.org/learn/applied-data-science-capstone/peer/60zST/capstone-project-the-battle-of-neighborhoods-week-1).

<hr>

Copyright &copy; 2020 [sathwik vadlamani](https://nikhil1006.github.io). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).